In [3]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('data/news/dataset.csv')


#keep only the columns we need - title and date
df = df[['Title', 'Date']]

# Display the first few rows of the DataFrame
df.head()

,Title,Date
0,"ATMs to become virtual bank branches, accept d...","May 26, 2020, Tuesday"
1,IDFC First Bank seniors to forgo 65% of bonus ...,"May 26, 2020, Tuesday"
2,"Huge scam in YES Bank for many years, says Enf...","May 25, 2020, Monday"
3,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...","May 24, 2020, Sunday"
4,DCB Bank's profit before tax declines 37.6% to...,"May 23, 2020, Saturday"


In [8]:
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Filter the DataFrame for dates between 2018 and 2020
filtered_df = df[(df['Date'] >= '2018-01-01') & (df['Date'] <= '2020-12-31')]

# Display the first few rows of the filtered DataFrame
filtered_df.head()

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('filtered_dataset.csv', index=False)

# Display the shape of the filtered DataFrame
print(filtered_df.shape)

(4726, 2)


In [ ]:
import pandas as pd
from transformers import pipeline
import spacy
# from textblob import TextBlob

# Load the FinBERT sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")


# Load your dataset
df = pd.read_csv("data/news/filtered_dataset.csv", names=["Title", "Date"])

# Apply the sentiment analysis model to the 'Title' column and get sentiment scores
df['Sentiment'] = df['Title'].apply(lambda x: sentiment_pipeline(x)[0]['label'])
# df['Sentiment_Score'] = df['Title'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Extract named entities
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

df['Entities'] = df['Title'].apply(extract_entities)

# Display the enriched DataFrame
print(df.head())

In [ ]:
import openai

# Set up your OpenAI API key
openai.api_key = 'your-api-key-here'

# Define a function to get a response from OpenAI API
def get_openai_response(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100
    )
    return response.choices[0].text.strip()

# Loop over the top 10 headlines in the DataFrame
for headline in df['Title'].head(10):
    prompt = f"""
    Extract the following features from the news headline:
    1. Named Entities
    2. Topic
    3. Sector/Industry
    4. Financial Metrics
    5. Tone

    Headline: "{headline}"
    """
    response = get_openai_response(prompt)
    print(f"Headline: {headline}\nResponse: {response}\n")

| **News** | **Sentiment** | **Named Entities** | **Topic** | **Sector/Industry** | **Financial Metrics** | **Tone** |
|-------------------------------------------|----------------|-----------------------------|-------------------------|------------------------|--------------------------|------------------|
| NBFC yields yet to show contraction...    | Neutral        | NBFC, Govt                  | Spreads                  | Finance                 | None                     | Neutral          |
| Sitharaman gives liquidity boost...       | Positive       | Sitharaman, Shadow Banks, MFIs | Liquidity Boost         | Finance                 | Rs 75,000 cr             | Optimistic       |
| Kotak Bank profit before tax slips...     | Negative       | Kotak Bank                  | Earnings Report          | Banking                 | 10.6% decrease, Rs 2,674 cr | Cautionary    |
| Differentiated bank holdco norms...       | Positive       | Bandhan, IDFC First          | Regulation Change        | Banking                 | None                     | Optimistic       |
| Bad bank may start with Rs 60K-crore...   | Neutral        | Bad Bank, Govt               | NPA Management           | Banking                 | Rs 60K-crore NPAs         | Neutral          |
| Some MSMEs may need deep restructuring... | Negative       | MSMEs, Union Bank            | Restructuring            | Finance                 | None                     | Cautionary       |
| IBA considering proposal to set up...     | Positive       | IBA, PSBs                   | NPA Management           | Banking                 | None                     | Optimistic       |
| Not offering any emergency loan...        | Neutral        | SBI, YONO                   | Clarification            | Banking                 | None                     | Neutral          |
